# Topics in Econometrics and Data Science: Tutorial 6

#### General Note

You will very likely find the solution to these exercises online. We, however, strongly encourage you to work on these exercises without doing so. Understanding someone else’s solution is very different from coming up with your own. Use the lecture notes and try to solve the exercises independently.

# Section 1 cont'd: Basic Statistics

## Exercise 1: Hypothesis Testing (II)
Now we want to test whether the proportion (probability of success) in a group equals a certain given value.

1. Use `help(stats.binomtest)` to make yourself familiar with the [`binomtest`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.binomtest.html) function.

In [63]:
import scipy.stats as stats
help(stats.binomtest)

Help on function binomtest in module scipy.stats._binomtest:

binomtest(k, n, p=0.5, alternative='two-sided')
    Perform a test that the probability of success is p.
    
    The binomial test [1]_ is a test of the null hypothesis that the
    probability of success in a Bernoulli experiment is `p`.
    
    Details of the test can be found in many texts on statistics, such
    as section 24.5 of [2]_.
    
    Parameters
    ----------
    k : int
        The number of successes.
    n : int
        The number of trials.
    p : float, optional
        The hypothesized probability of success, i.e. the expected
        proportion of successes.  The value must be in the interval
        ``0 <= p <= 1``. The default value is ``p = 0.5``.
    alternative : {'two-sided', 'greater', 'less'}, optional
        Indicates the alternative hypothesis. The default value is
        'two-sided'.
    
    Returns
    -------
    result : `~scipy.stats._result_classes.BinomTestResult` instance
      

2. A poll taken in $2003$ among $200$ Europeans found that only $16\%$ favored the policies of the United States. Perform a test of significance to see whether this is significantly different from the $50\%$ proportion of Americans in favor of these policies.

In [64]:
stats.binomtest(32,200,0.5) # two-sided test

BinomTestResult(k=32, n=200, alternative='two-sided', statistic=0.16, pvalue=1.818080038171794e-23)

3. A new drug therapy is tested. Of $50$ patients in the study, $40$ had no recurrence in their illness after $18$ months. With no drug therapy, the expected percentage of no recurrence would have been $75\%$. Does the data support the hypothesis that this percentage has increased? What is the $p$-value? Why is useful to consider a one-sided test?

In [65]:
stats.binomtest(40,50,0.75) # two-sided test

BinomTestResult(k=40, n=50, alternative='two-sided', statistic=0.8, pvalue=0.514121165666749)

In [66]:
stats.binomtest(40,50,0.75, alternative = "greater") # One-sided hypothesis

# We want to know whether the drug therapy increases the proportion of patients with no recurrence. 
# A two-sided test would also consider the possibility of a decrease, which is not relevant here. 
# A one-sided test is more powerful for detecting an increase in the percentage.

BinomTestResult(k=40, n=50, alternative='greater', statistic=0.8, pvalue=0.2622023101895092)

## Exercise 2: Bootstrap
Load the [`lightbulb_lifetime.csv`](https://maramattes.github.io/StatProg-HHU/data/lightbulb_lifetime.csv) data, which contains a (simulated) sample of lifespans of lightbulbs measured in hours. 

In [ ]:
import pandas as pd
import numpy as np
import os
path = ".../data/"
os.chdir(path)
lightbulb_lifetime = pd.read_csv('lightbulb_lifetime.csv', sep=';', na_values=".")
lightbulb_lifetime.head()

,0
0,572.691989
1,26.268241
2,797.757928
3,571.500317
4,545.361518


1. Estimate the median lifetime of a lightbulb.\
**Hint:** you can use `.values` to convert pandas dataframes into arrays and `.flatten()` to create vectors from arrays.

In [68]:
lightbulb_lifetime = lightbulb_lifetime.values
lightbulb_lifetime = lightbulb_lifetime.flatten()
# lightbulb_lifetime

In [69]:
med = np.median(lightbulb_lifetime)
print(med)

596.2420166828964


2. The sample median is an asymptotically normal distributed estimator for the median. Try to estimate the variance of the sample median with bootstrap.  
**Hint**: Use `np.random.choice()`, additionally `np.empty()` could be helpful.

In [70]:
import numpy as np
import math
import scipy.stats as stats

np.random.seed(0)

n = 200
B = 500

bootstrap_sample1 = np.empty((B, n))
for j in range(B):
     bootstrap_sample1[j] = np.random.choice(lightbulb_lifetime,n)

# or even better
bootstrap_sample = np.random.choice(lightbulb_lifetime,(B,n))
bootstrap_est = np.median(bootstrap_sample,axis = 1)
#print(bootstrap_est)

In [71]:
var_est = np.var(bootstrap_est)
print(var_est)

2800.3657745531796


$$
\widehat{med}_n \pm \hat{\sigma}_{boot} q(1- \frac{\alpha}{2})
$$

In [72]:
alpha = 0.05
quantile = stats.norm.ppf(q = 1-alpha/2)
# using the asymptotic normal distribution
confidence_interval1 = (med - np.sqrt(var_est)*quantile,med + np.sqrt(var_est)*quantile)
# using the sample quantiles for the estimator
confidence_interval2 = np.quantile(bootstrap_est, (alpha/2,1-alpha/2))
print(confidence_interval1)
print(confidence_interval2)

(492.5236971543423, 699.9603362114506)
[538.31503576 706.09709512]


#### Here's the second variant (working on pandas DataFrames): 

1. Estimate the median lifetime of a lightbulb.

In [73]:
import pandas as pd
import numpy as np
lightbulb_lifetime = pd.read_csv('lightbulb_lifetime.csv', sep=';', na_values=".")
lightbulb_lifetime.head()

,0
0,572.691989
1,26.268241
2,797.757928
3,571.500317
4,545.361518


In [74]:
med = np.median(lightbulb_lifetime)
print(med)

596.2420166828964


2. The sample median is an asymptotically normal distributed estimator for the median. Try to estimate the variance of the sample median with bootstrap.  
**Hint**: Use `np.random.choice()`, additionally `np.empty()` could be helpful.

In [75]:
import numpy as np
import math
import scipy.stats as stats
import random

np.random.seed(0)
n = 200
B = 500
alpha = 0.05
quantile = stats.norm.ppf(q = 1-alpha/2)

bootstrap_est = np.empty(B)

for j in range(B):
    indx = np.random.randint(n, size=n)
    bootstrap_sample2 = lightbulb_lifetime.iloc[indx]
    bootstrap_est[j] = np.median(bootstrap_sample2, axis = 0)

C:\Users\Mara Mattes\AppData\Local\Temp\ipykernel_23440\1913863860.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  bootstrap_est[j] = np.median(bootstrap_sample2, axis = 0)


In [76]:
var_est = np.var(bootstrap_est)
# using the asymptotic normal distribution
confidence_interval1 = (med - np.sqrt(var_est)*quantile,med + np.sqrt(var_est)*quantile)

# using the sample quantiles for the estimator
confidence_interval2 = np.quantile(bootstrap_est, (alpha/2, 1-alpha/2))

print(confidence_interval1)
print(confidence_interval2)

(491.9441320151656, 700.5399013506271)
[539.14526642 706.09709512]


3. Show with a simulation that your confidence interval is able to asymptotically satisfy the confidence level. Use standard normal distributed and exponential ( with parameter 0.01) distributed random variables in your simulation.  
**Hint**:`np.random.normal()` and `np.random.exponential()`

In [77]:
n = 500
B = 200
l = 10000

In [ ]:
# normal distribution
np.random.seed(123)
mu = 0
sigma = 1
#the true median is zero

alpha = 0.05
quantile = stats.norm.ppf(q = 1-alpha/2)
count = 0
for i in range(0, l):
    sample = np.random.normal(mu, sigma, n)
    med_est = np.median(sample)
    bootstrap_sample = np.random.choice(sample,(B,n))
    bootstrap_est = np.median(bootstrap_sample,axis = 1)
    var_est = np.var(bootstrap_est)
    confidence_interval1 = (med_est - np.sqrt(var_est)*quantile,med_est + np.sqrt(var_est)*quantile)
    if confidence_interval1[0] > 0 or confidence_interval1[1] < 0:
        count += 1
print("percentage of rejected null hypothesis:", count/l*100)

percentage of rejected null hypothesis: 5.48


In [ ]:
# exponential distribution
np.random.seed(123)
lam = 1 / 0.01
med = np.log(2) / 0.01 #true median

alpha = 0.05
quantile = stats.norm.ppf(q = 1-alpha/2)
count = 0
for i in range(0, l):
    sample = np.random.exponential(lam, n)    
    med_est = np.median(sample)
    bootstrap_sample = np.random.choice(sample,(B,n))
    bootstrap_est = np.median(bootstrap_sample,axis =1)
    var_est = np.var(bootstrap_est)
    confidence_interval1 = (med_est - np.sqrt(var_est)*quantile,med_est + np.sqrt(var_est)*quantile)
    if confidence_interval1[0] > med or confidence_interval1[1] < med:
        count += 1
print("percentage of rejected null hypothesis:", count/l*100)

percentage of rejected null hypothesis: 5.94
